<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSFB-Assignment-4" data-toc-modified-id="DSFB-Assignment-4-1">DSFB Assignment 4</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#DonorsChoose.org" data-toc-modified-id="DonorsChoose.org-1.0.1">DonorsChoose.org</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-1.0.2">Data</a></span></li><li><span><a href="#Special-NLP-Libraries" data-toc-modified-id="Special-NLP-Libraries-1.0.3">Special NLP Libraries</a></span></li></ul></li><li><span><a href="#IMPORTS" data-toc-modified-id="IMPORTS-1.1">IMPORTS</a></span></li></ul></li><li><span><a href="#PART-1:-Prep" data-toc-modified-id="PART-1:-Prep-2">PART 1: Prep</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Download-the-Data" data-toc-modified-id="Download-the-Data-2.0.1">Download the Data</a></span></li></ul></li></ul></li><li><span><a href="#PART-2:-Load-Data" data-toc-modified-id="PART-2:-Load-Data-3">PART 2: Load Data</a></span></li><li><span><a href="#PART-3:-Preprocess-Text" data-toc-modified-id="PART-3:-Preprocess-Text-4">PART 3: Preprocess Text</a></span></li><li><span><a href="#PART-4:--Make-an-LDA-topic-model-for-the-ESSAYS." data-toc-modified-id="PART-4:--Make-an-LDA-topic-model-for-the-ESSAYS.-5">PART 4:  Make an LDA topic model for the ESSAYS.</a></span></li><li><span><a href="#PART-5:--Make-an-LDA-topic-model-for-the-DESCRIPTIONS." data-toc-modified-id="PART-5:--Make-an-LDA-topic-model-for-the-DESCRIPTIONS.-6">PART 5:  Make an LDA topic model for the DESCRIPTIONS.</a></span></li></ul></div>

# DSFB Assignment 4

In this assignment, you will begin to work with text data and natural language processing. You will analyze aspects of th DonorsChoose.org program. Aspects of this project were first posed as a Kaggle challenge and the data comes from [Kaggle DonorsChoose.org Application Screening challenge](https://www.kaggle.com/c/donorschoose-application-screening/data). We have changed the nature of what you need to do in this assignment (so it does not track what was done in the Kaggle Challenge), but nevertheless using or referring to the Kaggle Challenge repository is not allowed for the assignment.

###  DonorsChoose.org  
  
Founded in 2000 by a high school teacher in the Bronx, DonorsChoose.org empowers public school teachers from across the country to request much-needed materials and experiences for their students. At any given time, there are thousands of classroom requests that can be brought to life with a gift of any amount. DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the DonorsChoose.org website. In this assignment, you will analyze the text of the essays and requirements from each proposal.

<img src="https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg" width="500" height="500" align="center"/>

Image source: https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg

### Data

As you will see, this dataset includes many different kinds of features with structured and unstructured data. The dataset consists of application materials (see *application_data.csv*) and resources requested (see *resource_data.csv*). The application materials (see *application_data.csv*) contain the following features.

| Feature name  | Description  |
|----------------|--------------|
| id  | Unique id of the project application    |
| teacher_id    | id of the teacher submitting the application  |
| teacher_prefix    | title of the teacher's name (Ms., Mr., etc.)    |
| school_state    | US state of the teacher's school    |
| project_submitted_datetime    | application submission timestamp    |
| project_grade_category    | school grade levels (PreK-2, 3-5, 6-8, and 9-12)   |
| project_subject_categories   | category of the project (e.g., "Music & The Arts")    |
| project_subject_subcategories    | sub-category of the project (e.g., "Visual Arts")    |
| project_title    | title of the project    |
| project_essay_1    | first essay*   |
| project_essay_2    | second essay*    |
| project_essay_3    | third essay*   |
| project_essay_4    | fourth essay*  |
| project_resource_summary    | summary of the resources needed for the project    |
| teacher_number_of_previously_posted_projects   | number of previously posted applications by the submitting teacher    |
| project_is_approved    | whether DonorsChoose proposal was accepted (0="rejected", 1="accepted"); train.csv only    |


\*Note: Prior to May 17, 2016, the prompts for the essays were as follows:

  * project_essay_1: "Introduce us to your classroom"  

  * project_essay_2: "Tell us more about your students"  

  * project_essay_3: "Describe how your students will use the materials you're requesting"  

  * project_essay_4: "Close by sharing why your project will make a difference"  

Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:

  * project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."  

  * project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"  

For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be missing (i.e. NaN).


### Special NLP Libraries

We will use several new libraries for this assignment - so be sure to first install those on your machine by with `pip` in a terminal:

    pip install --user -U nltk
    pip install -U gensim
    pip install -U spacy
    pip install -U pyldavis

## IMPORTS

In [1]:
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

#BeautifulSoup
from bs4 import BeautifulSoup

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# Special Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

# Set some CONSTANTS that will be used later
SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

# PART 1: Prep

**PROBLEM**: To use a particular model in the `spacy` package, you need to manually download and install that particular model. You will need to run the following code from a terminal: `python -m spacy download en_core_web_sm`. Rather than doing that manually from bash in a separate terminal program, do it inline below using a "magic" command in jupyter. HINT: Use *!* followed by a bash command in a cell to run a bash command.

In [2]:
# Download en_core_web_sm for spacy
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


**PROBLEM**: To confirm that `spacy` is working (and `en_core_web_sm` is installed on your computer), you should be able to use `spacy.load()` to build a `Language` object to perform some basic nlp. Do that below:

**Solution:**

The following doesn't work on Mac with jupyter notebook: **nlp = spacy.load("en_core_web_sm")**

So I used the approach described here: [source](https://github.com/explosion/spaCy/issues/4577) 

In [2]:
# Test use of spacy by using the spacy.load() function
import en_core_web_sm
nlp = en_core_web_sm.load()

**PROBLEM**: Use nltk.download() to download a list of raw stopwords. (see NLTK documentation)

In [21]:
# Download NLTK stopwords
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

**PROBLEM**: Use the `stopwords` object from `nltk` to build a list of English stopwords. 

In [3]:
# Get English Stopwords from NLTK
stop_words = set(stopwords.words('english')) 

**PROBLEM**: Extend your `stop_words` list with some additional stopwords that you believe should be ignored in this particular context.

In [4]:
# Extend the stop word list  
# We remove words like student, school etc because they are present in every essay and don't provide meaningful information
stop_words = stop_words.union({'hello','name','mrs','mr','ms','miss','classroom'\
                               'students','teacher','project','difference','school'\
                               'come','th','also','teach','u','could','would','from'})

### Download the Data

Unlike other projects, this project includes a training set too big for GitHub. Through the terminal lab of Jupyter lab, download the data using the *wget* command, unzip it using the *zip* command and check that it's in the root directory of the project. 

Locations : 

    Applications dataset: https://storage.googleapis.com/dsfm/application/application_data.csv.zip
    Resources dataset: https://storage.googleapis.com/dsfm/application/resource_data.csv.zip
    
Hint: Use *wget* and *unzip* commands. Use *!* followed by a bash command in a cell to run a bash command.

**PROBLEM**: wget the data

**Solution:**
I am using curl command line to download from the web because I have a mac

In [6]:
# wget the data
!curl https://storage.googleapis.com/dsfm/application/application_data.csv.zip curl --output application_data.csv.zip
!curl https://storage.googleapis.com/dsfm/application/resource_data.csv.zip --output resource_data.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  107M  100  107M    0     0  2222k      0  0:00:49  0:00:49 --:--:-- 3222k 0:00:55  0:00:07  0:00:48 1853k
curl: (6) Could not resolve host: curl
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.4M  100 40.4M    0     0  3571k      0  0:00:11  0:00:11 --:--:-- 3738k  0  2823k      0  0:00:14  0:00:02  0:00:12 2822k


**PROBLEM**: unzip the data

In [7]:
# unzip the data
!Unzip application_data.csv.zip
!Unzip resource_data.csv.zip

Archive:  application_data.csv.zip
  inflating: application_data.csv    
Archive:  resource_data.csv.zip
  inflating: resource_data.csv       


# PART 2: Load Data

**PROBLEM**: Load `application_data.csv` and investigate it a bit.

In [5]:
# Load applications
applications = pd.read_csv("application_data.csv")
applications.head(2)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0


In [6]:
applications.drop_duplicates()
applications.shape

(182080, 16)

In [7]:
applications.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved'],
      dtype='object')

In [8]:
applications.info(memory_usage=False, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 16 columns):
id                                              182080 non-null object
teacher_id                                      182080 non-null object
teacher_prefix                                  182076 non-null object
school_state                                    182080 non-null object
project_submitted_datetime                      182080 non-null object
project_grade_category                          182080 non-null object
project_subject_categories                      182080 non-null object
project_subject_subcategories                   182080 non-null object
project_title                                   182080 non-null object
project_essay_1                                 182080 non-null object
project_essay_2                                 182080 non-null object
project_essay_3                                 6374 non-null object
project_essay_4                        

* We see that we have 182080 entries and 16 features. 
* 4 Null values are present in the prefix field.
* 175706 Null values are present in the project_essay_3 & project_essay_4 which we can explain by the fact that for all projects submitted after 2016-05-17, these fields disappeared.

**PROBLEM**: Load `resource_data.csv` and investigate it a bit.

In [9]:
# Load resources
resources = pd.read_csv("resource_data.csv")
resources.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [10]:
resources.columns

Index(['id', 'description', 'quantity', 'price'], dtype='object')

In [11]:
resources.drop_duplicates()
resources.shape

(1541272, 4)

In [12]:
len(resources['id'].unique())

260115

* We see that we have 260115 unique ids and 1541272 entries. This make sense because the same teacher can request more than one item.

In [13]:
resources.info(memory_usage=False, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541272 entries, 0 to 1541271
Data columns (total 4 columns):
id             1541272 non-null object
description    1540980 non-null object
quantity       1541272 non-null int64
price          1541272 non-null float64
dtypes: float64(1), int64(1), object(2)

* We see that some of the ressources have their field description as Null.

**PROBLEM**: Some of the essays are NA. Replace NAs with empty strings.

In [14]:
# Replace NaN values in essay columns with ''
applications[['project_essay_3','project_essay_4']] = \
                                    applications[['project_essay_3','project_essay_4']].fillna(value='')

**PROBLEM**: To simplify matters, combine all essays into just one feature called "essays"

In [15]:
# Combine essays
applications['essays']=applications['project_essay_1']+' '+applications['project_essay_2']+' '+\
                        applications['project_essay_3']+' '+applications['project_essay_4']
applications.drop(labels=['project_essay_1','project_essay_2','project_essay_3','project_essay_4'],axis=1, inplace=True)

**PROBLEM**: Merge the resources and application datasets on the *id* feature.

In [16]:
#We first preprocess the description column to replace NaN values with empty string
resources['description']=resources['description'].fillna(value='')
# Merge two datasets
applications_merged = applications.merge(resources, on='id', how='left')
# Check the data to confirm it worked
print("We have {} applications in total".format(applications_merged.shape[0]))
applications_merged.head(2)


We have 1081830 applications in total


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essays,description,quantity,price
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,My students need 6 Ipod Nano's to create and d...,26,1,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...,3,149.99
1,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,My students need 6 Ipod Nano's to create and d...,26,1,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...,3,149.99


**PROBLEM**: Keep the following data for additional analysis (the id and the text features): `id`, `school_state`, `project_subject_categories`, `project_subject_subcategories`, `essays`, `description`

In [17]:
FEATURE_NAMES = ['id','school_state', 'project_subject_categories', 'project_subject_subcategories', 'essays', 'description']

In [18]:
# Keep the Text Features
applications_merged = applications_merged[FEATURE_NAMES]

# PART 3: Preprocess Text

Make an independent copy of the data so we can restart here when testing...

In [19]:
data = copy.copy(applications_merged)

**PROBLEM**: Define a custom function `clean_punctuation()` to remove some punctuation from your text data. You don't have to do absolutely everything one might want to do - just show that you can do it. Start with each some easy operations with `str.replace()`.

In [20]:
# Define a custom function to clean punctuation from  given text
def clean_punctuation(txt):
    txt = txt.replace('.','').replace(',','').replace(';','').replace('\'','')\
            .replace('\"','').replace('!','').replace('?','').replace('(','')\
            .replace(')','').replace(':','').replace('-','').replace('_','').replace('&','')
    return txt

**PROBLEM**: Use the `apply()` function from pandas to _apply_ that function down the `essays` column of your data.

**Solution**


In [21]:
# Apply your function to clean the essays column
FEATURES_TO_CLEAN = ['school_state', 'project_subject_categories', 'project_subject_subcategories', 'essays', 'description']
for feature in FEATURES_TO_CLEAN:
    print("cleaning ", feature)
    data[feature] = data[feature].apply(clean_punctuation)  
data.head()

cleaning  school_state
cleaning  project_subject_categories
cleaning  project_subject_subcategories
cleaning  essays
cleaning  description


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy Language,Literacy,Most of my kindergarten students come from low...,Apple iPod nano� 16GB MP3 Player 8th Generati...
1,p036502,NV,Literacy Language,Literacy,Most of my kindergarten students come from low...,Apple iPod nano� 16GB MP3 Player 8th Generati...
2,p039565,GA,Music The Arts Health Sports,Performing Arts Team Sports,Our elementary school is a culturally rich sch...,Reebok Girls Fashion Dance Graphic TShirt Dd ...
3,p233823,UT,Math Science Literacy Language,Applied Sciences Literature Writing,Hello\r\nMy name is Mrs Brotherton I teach 5th...,3doodler Start Full Edu Bundle
4,p185307,NC,Health Sports,Health Wellness,My students are the greatest students but are ...,BALL PG 4 POLY SET OF 6 COLORS


**PROBLEM**: Define **another** custom function called `clean_re()` to clean your text data using regular expressions. Do at least two "cleanings" (i.e., show that you can use the `re` library).

In [22]:
#We will work with a random subset of 10'000 entries, as our dataset is too large to be processed efficiently 
data = data.sample(n=10000, random_state=SEED)
data.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
556546,p060918,NY,Health Sports,Health Wellness,I work in an amazing school with the best stud...,Nabisco Ritz Bits Cheese Cracker Sandwiches 1 ...
592611,p212662,AR,Math Science Literacy Language,Health Life Science Literacy,There are 100 kindergarten students in our sch...,LL714 Level D Nonfiction Extra Book Set KGr 1
494454,p200812,IN,Math Science,Applied Sciences Mathematics,I work with twentyseven amazing students that ...,ZOOB 250 Piece Building Set
68768,p075358,UT,Health Sports,Gym Fitness Team Sports,We will have brand new students in our brand n...,Tachikara VolleyAll Indoor Volleyball
401690,p236494,FL,Literacy Language Math Science,Literacy Mathematics,I work in a Title One school in which 99% of o...,Master Lock 1500iD Speed Dial Combination Lock...


In [23]:
# Define a custom function to clean some given text
def clean_re(txt):
    #First let's Remove leading and trailing whitespaces from the strings
    txt = txt.strip()
    # Remove HTML using BeautifulSoup
    txt = BeautifulSoup(txt).get_text()
    #We replace all new lines, tabs, whitespaces with a single whitespace using regex
    txt = re.sub(r'\\n+',' ',txt)
    txt = re.sub(r'\\t+',' ',txt)
    txt = re.sub(r'\\r+',' ',txt)
    txt = re.sub(r"\s+",' ',txt)
    # Remove non-letters
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    return txt

In [24]:
# Apply clean_re() to all features
for feature in FEATURES_TO_CLEAN:
    print("cleaning ", feature)
    data[feature] = data[feature].apply(clean_re)
    data[feature] = data[feature].str.lower()
data.head()

cleaning  school_state
cleaning  project_subject_categories
cleaning  project_subject_subcategories
cleaning  essays
cleaning  description


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
556546,p060918,ny,health sports,health wellness,i work in an amazing school with the best stud...,nabisco ritz bits cheese cracker sandwiches ...
592611,p212662,ar,math science literacy language,health life science literacy,there are kindergarten students in our sch...,ll level d nonfiction extra book set kgr
494454,p200812,in,math science,applied sciences mathematics,i work with twentyseven amazing students that ...,zoob piece building set
68768,p075358,ut,health sports,gym fitness team sports,we will have brand new students in our brand n...,tachikara volleyall indoor volleyball
401690,p236494,fl,literacy language math science,literacy mathematics,i work in a title one school in which of o...,master lock id speed dial combination lock...


**PROBLEM**: Remove stopwords. (Hint: use stopwords from nltk's `stopwords()` plus any additions you'd like to make. Then, again, define a custom function and then apply it to all features.)

In [25]:
# Define custom function to remove stopwords
def remove_stopwords(txt):
    # Split into individual words
    word_tokens = txt.split()
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    txt = ' '.join(filtered_sentence)
    return txt

In [26]:
# Apply function to remove stopwords  
for feature in FEATURES_TO_CLEAN:
    print("cleaning ", feature)
    data[feature] = data[feature].apply(remove_stopwords)  
data.head()

cleaning  school_state
cleaning  project_subject_categories
cleaning  project_subject_subcategories
cleaning  essays
cleaning  description


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
556546,p060918,ny,health sports,health wellness,work amazing school best students ever always ...,nabisco ritz bits cheese cracker sandwiches oz...
592611,p212662,ar,math science literacy language,health life science literacy,kindergarten students school come high poverty...,level nonfiction extra book set kgr
494454,p200812,,math science,applied sciences mathematics,work twentyseven amazing students want learn s...,zoob piece building set
68768,p075358,ut,health sports,gym fitness team sports,brand new students brand new wing school start...,tachikara volleyall indoor volleyball
401690,p236494,fl,literacy language math science,literacy mathematics,work title one school students receive free re...,master lock id speed dial combination lock ass...


**PROBLEM**: Now use Gensim’s `simple_preprocess()` function to tokenize and clean up your text data. TIP: `simple_preprocess()` returns a list of words, so we want to wrap it with a function that joins the list back together into a string.

In [27]:
# Define custom function to wrap simple_preprocess() from gensim
def process_gensim(txt):
    #Use simple_preprocess() to lowercase and tokenize the text
    tokens = simple_preprocess(txt)
    return tokens

# Define a function that joins back the list of words into a string
def join_tokens(tokens):
    txt = ' '.join(tokens)
    return txt

In [28]:
# Apply simple_preprocess() to all features
for feature in FEATURES_TO_CLEAN:
    print("cleaning ", feature)
    data[feature] = data[feature].apply(process_gensim)  
data.head()

cleaning  school_state
cleaning  project_subject_categories
cleaning  project_subject_subcategories
cleaning  essays
cleaning  description


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
556546,p060918,[ny],"[health, sports]","[health, wellness]","[work, amazing, school, best, students, ever, ...","[nabisco, ritz, bits, cheese, cracker, sandwic..."
592611,p212662,[ar],"[math, science, literacy, language]","[health, life, science, literacy]","[kindergarten, students, school, come, high, p...","[level, nonfiction, extra, book, set, kgr]"
494454,p200812,[],"[math, science]","[applied, sciences, mathematics]","[work, twentyseven, amazing, students, want, l...","[zoob, piece, building, set]"
68768,p075358,[ut],"[health, sports]","[gym, fitness, team, sports]","[brand, new, students, brand, new, wing, schoo...","[tachikara, volleyall, indoor, volleyball]"
401690,p236494,[fl],"[literacy, language, math, science]","[literacy, mathematics]","[work, title, one, school, students, receive, ...","[master, lock, id, speed, dial, combination, l..."


**PROBLEM**: Lemmatize the text. (Hint: Define a custom function and then apply it to all features.)

In [29]:
# Write a lemmatization function based on nltk.stem.WordNetLemmatizer()
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer() 
def process_lemmatize(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(x) for x in tokens]
    return lemmatized_tokens

In [30]:
# Apply lemmatize_text() to all features  
for feature in FEATURES_TO_CLEAN:
    print("cleaning ", feature)
    data[feature] = data[feature].apply(process_lemmatize) 
    
#Put back the list of words to string if needed
#for feature in FEATURES_TO_CLEAN:
#    help_[feature] = help_[feature].apply(join_tokens) 
    
data.head()

cleaning  school_state
cleaning  project_subject_categories
cleaning  project_subject_subcategories
cleaning  essays
cleaning  description


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
556546,p060918,[ny],"[health, sport]","[health, wellness]","[work, amazing, school, best, student, ever, a...","[nabisco, ritz, bit, cheese, cracker, sandwich..."
592611,p212662,[ar],"[math, science, literacy, language]","[health, life, science, literacy]","[kindergarten, student, school, come, high, po...","[level, nonfiction, extra, book, set, kgr]"
494454,p200812,[],"[math, science]","[applied, science, mathematics]","[work, twentyseven, amazing, student, want, le...","[zoob, piece, building, set]"
68768,p075358,[ut],"[health, sport]","[gym, fitness, team, sport]","[brand, new, student, brand, new, wing, school...","[tachikara, volleyall, indoor, volleyball]"
401690,p236494,[fl],"[literacy, language, math, science]","[literacy, mathematics]","[work, title, one, school, student, receive, f...","[master, lock, id, speed, dial, combination, l..."


**PROBLEM**: What happened to the data in the pandas dataframe

ANSWER: In was converted from long text into a list of individual words.

# PART 4:  Make an LDA topic model for the ESSAYS.

Define an LDA topic model for the `essays`. Compute the "Coherence score." Visually inspect the topic model by inspecting the top keywords from each model. Gensim provides functions for all of these tasks.  

In [45]:
#We use the essays column from our dataset after lemmatization
essays = data['essays'].to_list()

# Create Dictionary
id2word = corpora.Dictionary(essays)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in essays]

# View result for the first essay
# Human readable format of corpus (term-frequency) of the first essay
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[0:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, #10 topics seems fairly reasonable for this problem
                                           random_state=SEED,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=essays, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46868928998946047


If you use gensim and the following three variables, then you can visualize topics & keywords with the code below.

    lda_model:    this is an LDA model generated by gensim.models.ldamodel.LdaModel()
    id2word:      this is the dictionary term IDs from corpora.Dictionary()
    corpus:       this is the collection of "documents"


In [46]:
# Visualize topics-keywords
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.237967 -0.055455       1        1  44.841999
2     -0.207591 -0.038780       2        1  16.171412
0     -0.147383  0.105447       3        1  11.748345
1     -0.105317  0.042377       4        1   8.330415
5     -0.144992 -0.004200       5        1   7.003715
8     -0.033003 -0.217662       6        1   4.979972
3      0.096085  0.355088       7        1   3.587159
6      0.226401 -0.171855       8        1   1.918276
4      0.291062 -0.011132       9        1   0.753966
9      0.262706 -0.003828      10        1   0.664744, topic_info=     Category          Freq             Term         Total  loglift  logprob
101   Default  15956.000000             book  15956.000000  30.0000  30.0000
156   Default  12244.000000          reading  12244.000000  29.0000  29.0000
80    Default  81319.000000          student  81319.000000  28.0000  28.0000
51    Default  16184.000000         learning  16184.000000  27.0000  27.0000
75    Default  25575.000000           school  25575.000000  26.0000  26.0000
155   Default   6424.000000             read   6424.000000  25.0000  25.0000
531   Default   4071.000000           supply   4071.000000  24.0000  24.0000
98    Default   3811.000000              art   3811.000000  23.0000  23.0000
164   Default   7070.000000            skill   7070.000000  22.0000  22.0000
782   Default   6701.000000         material   6701.000000  21.0000  21.0000
195   Default  16699.000000        classroom  16699.000000  20.0000  20.0000
136   Default   3829.000000         language   3829.000000  19.0000  19.0000
183   Default   4166.000000         activity   4166.000000  18.0000  18.0000
1016  Default   2595.000000          special   2595.000000  17.0000  17.0000
470   Default   3761.000000          library   3761.000000  16.0000  16.0000
39    Default   5957.000000            grade   5957.000000  15.0000  15.0000
262   Default   8475.000000             love   8475.000000  14.0000  14.0000
17    Default   7088.000000              day   7088.000000  13.0000  13.0000
137   Default   4707.000000            level   4707.000000  12.0000  12.0000
205   Default   2134.000000             game   2134.000000  11.0000  11.0000
42    Default  13117.000000             help  13117.000000  10.0000  10.0000
1105  Default   2208.000000             play   2208.000000   9.0000   9.0000
178   Default   6565.000000              use   6565.000000   8.0000   8.0000
197   Default   3893.000000           create   3893.000000   7.0000   7.0000
306   Default   4330.000000             math   4330.000000   6.0000   6.0000
59    Default  12962.000000             need  12962.000000   5.0000   5.0000
754   Default   2974.000000           reader   2974.000000   4.0000   4.0000
50    Default  11816.000000            learn  11816.000000   3.0000   3.0000
200   Default   2576.000000          english   2576.000000   2.0000   2.0000
484   Default   3151.000000       technology   3151.000000   1.0000   1.0000
462    Topic1   4571.843750            every   4572.721680   0.8018  -4.8840
642    Topic1   2403.432617             much   2404.310059   0.8017  -5.5270
785    Topic1   2129.650635           parent   2130.528076   0.8016  -5.6480
26     Topic1   1983.781738             even   1984.660034   0.8016  -5.7189
13     Topic1   7771.010254             come   7774.518066   0.8016  -4.3535
305    Topic1   1610.426392              low   1611.303955   0.8015  -5.9274
630    Topic1   1133.155151          college   1134.032715   0.8013  -6.2789
38     Topic1   1975.740723               go   1977.416870   0.8012  -5.7230
307    Topic1   1010.599304             mean   1011.476807   0.8012  -6.3934
68     Topic1    993.654541           really    994.532104   0.8011  -6.4103
475    Topic1    987.253357         possible    988.134399   0.8011  -6.4168
1114   Topic1    969.088501            going    969.966064   0.8011  -6.435

**Interpretation:**

We see that 10 topics seem to be a good fit for the essays. The bubbles are distributed along the four axes and are not so much overlapping.
Also the subjects of the different clusters are well defined when you look at the most relevant terms of each cluster.

We can interpret the topics found from the 30 most relevant terms as follows:
* topic 1 & 2: general school vocabulary, the bubble is large which makes sense because it's common words to use in the essays
* topic 3: concerns computer science and technology related projects
* topic 4: concerns projects related to literature
* topic 5: concerns projects related to language skills
* topic 6: concerns confortable class envorenment and health
* topic 7: concerns artistic and painting related projects
* topic 8: concerns games and sports related applications
* topic 9: concerns music and theature related projects
* topic 10: concerns special kids, and disability related projects

# PART 5:  Make an LDA topic model for the DESCRIPTIONS.

Using the same K (and any other hyperparameters from Part 4), recompute a model for Descriptions. Compare the two sets of results. Do they vary? How? Why? Explain what you find. 

In [47]:
#We use the descriptions column from our dataset after lemmatization
descriptions = data['description'].to_list()

# Create Dictionary
id2word = corpora.Dictionary(descriptions)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in essays]

# View result for the first essay
# Human readable format of corpus (term-frequency) of the first essay
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[0:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=SEED,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=essays, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4903551170692011


In [48]:
# Visualize topics-keywords
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.226127 -0.029563       1        1  46.372108
8      0.220893  0.016547       2        1  15.737269
7      0.176512  0.083325       3        1   8.581023
6      0.042192 -0.122146       4        1   7.440957
1      0.159971 -0.049502       5        1   4.980616
5     -0.102481  0.218529       6        1   4.755201
4     -0.166664  0.250851       7        1   3.273514
3     -0.207752 -0.217851       8        1   3.212225
9     -0.155041 -0.226319       9        1   2.836276
2     -0.193756  0.076128      10        1   2.810809, topic_info=     Category          Freq           Term         Total  loglift  logprob
8     Default  15394.000000           book  15394.000000  30.0000  30.0000
811   Default  79179.000000        student  79179.000000  29.0000  29.0000
266   Default  11813.000000        reading  11813.000000  28.0000  28.0000
301   Default  15879.000000       learning  15879.000000  27.0000  27.0000
298   Default   5887.000000          grade   5887.000000  26.0000  26.0000
748   Default   5817.000000          child   5817.000000  25.0000  25.0000
146   Default  11578.000000          learn  11578.000000  24.0000  24.0000
2144  Default   8027.000000           year   8027.000000  23.0000  23.0000
1086  Default   6198.000000           read   6198.000000  22.0000  22.0000
232   Default  24913.000000         school  24913.000000  21.0000  21.0000
295   Default  12599.000000           need  12599.000000  20.0000  20.0000
1193  Default   3727.000000            art   3727.000000  19.0000  19.0000
78    Default   3988.000000         supply   3988.000000  18.0000  18.0000
1084  Default   6913.000000          skill   6913.000000  17.0000  17.0000
2814  Default   3052.000000     technology   3052.000000  16.0000  16.0000
583   Default   6583.000000       material   6583.000000  15.0000  15.0000
2440  Default   6408.000000            use   6408.000000  14.0000  14.0000
517   Default   4278.000000           math   4278.000000  13.0000  13.0000
129   Default   8283.000000           love   8283.000000  12.0000  12.0000
206   Default   3628.000000        library   3628.000000  11.0000  11.0000
510   Default   3007.000000        science   3007.000000  10.0000  10.0000
108   Default   4168.000000       activity   4168.000000   9.0000   9.0000
74    Default  16288.000000      classroom  16288.000000   8.0000   8.0000
451   Default   3169.000000          first   3169.000000   7.0000   7.0000
1187  Default  12758.000000           help  12758.000000   6.0000   6.0000
11    Default   4597.000000          level   4597.000000   5.0000   5.0000
555   Default   2453.000000           play   2453.000000   4.0000   4.0000
1521  Default   4273.000000          group   4273.000000   3.0000   3.0000
666   Default   6921.000000            day   6921.000000   2.0000   2.0000
6987  Default   3758.000000         create   3758.000000   1.0000   1.0000
1186   Topic1   4454.993164          every   4455.894531   0.7683  -4.7025
314    Topic1   3300.605225         family   3301.506836   0.7682  -5.0024
903    Topic1   3293.570312           free   3294.471924   0.7682  -5.0046
673    Topic1   3068.361328          lunch   3069.262939   0.7682  -5.0754
537    Topic1   2414.877686           know   2415.779297   0.7681  -5.3149
860    Topic1   1926.095459             go   1926.997070   0.7680  -5.5410
7139   Topic1   1883.909424        receive   1884.874756   0.7680  -5.5632
1754   Topic1   2466.892822           take   2468.182861   0.7679  -5.2936
1130   Topic1   1572.363892           face   1573.265503   0.7679  -5.7440
50     Topic1   1435.283447           live   1436.185059   0.7678  -5.8352
4150   Topic1   1146.186157          never   1147.087769   0.7677  -6.0601
1421   Topic1   1138.776367        healthy   1139.677856   0.7677  -6.0666
4126   Topic1    981.878662      breakfast    982.780212   0.7676  -6.2148
2784   To

**Interpretation:**

We see that 10 topics seem to be a bad fit for the descriptions. The most relevant words for each cluster don't make much sense for the items.
* **Topics 1, 2, 3, 5, 10: don't make much sense**
* Topic 4: concerns projects related to literature
* Topic 6: concerns scientific projects
* Topic 7: concerns artistic projects
* Topic 9: concerns projects related to language skills

A possible explanation is that even if the essays are quite similar for the applications, the ressources requested for those "similar application" could be quite different.

A solution would be to take a larger num_topics, that way we will cluster the topics of the items descriptions into more relevant subsets.